<a href="https://colab.research.google.com/github/hasse910/IA_CRE/blob/main/ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip -q install aiohttp pyngrok

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import os
import asyncio

# Certifique-se de que userdata está importado, por exemplo:
from google.colab import userdata

# Configurações iniciais
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
os.environ['OLLAMA_ORIGINS'] = '*' # Ou 'https://*.ngrok-free.app'

# !!! Verifique se ngrok_key está sendo obtido corretamente !!!
# Se userdata.get('ngrok_key') retornar None, os comandos ngrok podem falhar silenciosamente
# ou se comportar de forma inesperada.
ngrok_key = userdata.get('ngrok_key')
print(f"--- DEBUG: Valor de ngrok_key carregado: '{ngrok_key}' ---") # LINHA DE DEBUG
if not ngrok_key:
    print("AVISO: Chave do ngrok (ngrok_key) não encontrada no userdata!")
    # Você pode querer parar aqui ou usar um valor padrão se aplicável,
    # ou prosseguir sabendo que o ngrok pode não funcionar.

# Evento para sinalizar que o servidor Ollama está pronto
ollama_ready_event = asyncio.Event()

async def run_process_wrapper(cmd, is_server=False, ready_signal_in_log=None,
                              event_to_set_on_ready=None, event_to_wait_for=None,
                              suppress_ngrok_key_in_cmd_print=True):
    """
    Executa um comando e gerencia a espera/sinalização de prontidão.
    """
    global ngrok_key # Acessa a variável global ngrok_key

    if event_to_wait_for:
        print(f">>> '{cmd[0]}' esperando pelo evento de prontidão...")
        await event_to_wait_for.wait()
        print(f">>> Evento recebido, iniciando '{cmd[0]}'.")

    cmd_display_parts = []
    if suppress_ngrok_key_in_cmd_print and ngrok_key:
        for part in cmd:
            if part == ngrok_key:
                cmd_display_parts.append("[NGROK_KEY_OCULTA]")
            else:
                cmd_display_parts.append(part)
    else:
        cmd_display_parts = list(cmd)
    cmd_display_name = " ".join(cmd_display_parts)

    print(f'>>> starting {cmd_display_name}')

    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(stream, stream_name_suffix): # Esta é a função interna de run_process_wrapper
      # cmd é acessado do escopo externo de run_process_wrapper
      # ngrok_key também é acessado do escopo global/externo

      stream_name = f"{cmd[0]} {stream_name_suffix}"
      is_ngrok_stderr_pipe = (cmd[0] == 'ngrok' and 'http' in cmd and stream_name_suffix == 'stderr')

      if is_ngrok_stderr_pipe:
          print(f"--- DEBUG PIPE: Iniciando pipe para ngrok stderr ---")

      async for line_bytes in stream:
          line = line_bytes.decode('utf-8', errors='replace').strip()

          # Lógica de formatação do log (pode manter a sua ou simplificar para debug)
          log_line_display = f"[{stream_name}]: {line}"
          # Se quiser re-adicionar a lógica de ocultar chave:
          # if suppress_ngrok_key_in_cmd_print and ngrok_key and ngrok_key in line:
          #     log_line_display = f"[{stream_name}]: [linha contendo NGROK_KEY ocultada]"
          # else:
          #     log_line_display = f"[{stream_name}]: {line}"

          if is_ngrok_stderr_pipe:
              print(f"--- DEBUG NGROK STDERR RAW: '{line}' ---") # Mostra a linha crua do ngrok stderr
              if "url=" in line: # Procura pela parte da URL
                  print(f"--- DEBUG NGROK URL LINE DETECTED: '{line}' ---")

          print(log_line_display) # Imprime a linha de log formatada para todos os processos

          # Lógica de sinalização de evento (mantenha como estava)
          if is_server and ready_signal_in_log and ready_signal_in_log in line:
              if event_to_set_on_ready and not event_to_set_on_ready.is_set():
                  print(f">>> SERVIDOR '{cmd[0]}' ESTÁ PRONTO! (viu: '{ready_signal_in_log}')")
                  event_to_set_on_ready.set()

      if is_ngrok_stderr_pipe:
          print(f"--- DEBUG PIPE: Finalizando pipe para ngrok stderr ---")

    # Estas tasks rodam até o processo terminar ou serem canceladas.
    stdout_task = asyncio.create_task(pipe(p.stdout, 'stdout'))
    stderr_task = asyncio.create_task(pipe(p.stderr, 'stderr'))

    try:
        # Espera o processo terminar e as tasks de pipe também.
        # Se uma task de pipe falhar, ou o processo terminar, gather retornará.
        await asyncio.gather(stdout_task, stderr_task)
        # Espera explicitamente pelo processo para garantir que o código de retorno seja registrado.
        await p.wait()
    except asyncio.CancelledError:
        print(f">>> Tarefa para '{cmd_display_name}' foi cancelada.")
        # Garante que o processo filho seja terminado se a task principal for cancelada
        if p.returncode is None: # Se o processo ainda estiver rodando
            print(f">>> Tentando terminar o processo '{cmd_display_name}'...")
            p.terminate()
            await p.wait() # Espera a terminação
        raise # Re-levanta CancelledError para que gather possa lidar com ela
    except Exception as e:
        print(f"Erro inesperado ao executar '{cmd_display_name}': {e}")
        if p.returncode is None:
            p.terminate()
            await p.wait()
    finally:
        # Garante que as tasks de pipe sejam canceladas se ainda estiverem rodando
        # (por exemplo, se p.wait() retornou mas as tasks de pipe não por algum motivo)
        if not stdout_task.done(): stdout_task.cancel()
        if not stderr_task.done(): stderr_task.cancel()
        # Espera que as tasks de pipe realmente terminem após o cancelamento
        await asyncio.gather(stdout_task, stderr_task, return_exceptions=True)

        print(f">>> '{cmd_display_name}' finalizado com código {p.returncode if p.returncode is not None else 'N/A (possivelmente cancelado)'}.")
        if is_server and event_to_set_on_ready and not event_to_set_on_ready.is_set():
            print(f"ATENÇÃO: Servidor '{cmd[0]}' terminou antes de sinalizar prontidão ou foi cancelado.")
            event_to_set_on_ready.set() # Desbloqueia outros para evitar deadlock, mesmo que falhem


async def main():
    global ngrok_key # Permite que main acesse a variável ngrok_key
    if not ngrok_key:
        print("Chave do ngrok não definida. O túnel ngrok não será iniciado corretamente.")
        # Decide se quer parar ou continuar sem ngrok. Para este exemplo, vamos prosseguir.

    # 1. Configurar o token do ngrok primeiro (comando que termina)
    # Só executa se ngrok_key estiver definida
    if ngrok_key:
        await run_process_wrapper(
            ['ngrok', 'config', 'add-authtoken', ngrok_key],
            suppress_ngrok_key_in_cmd_print=True
        )
    else:
        print("Pular configuração do authtoken do ngrok pois a chave não foi fornecida.")


    # 2. Iniciar todos os outros processos concorrentemente,
    #    mas com dependência de prontidão para ollama pull e ngrok.
    try:
        await asyncio.gather(
            run_process_wrapper(
                ['ollama', 'serve'],
                is_server=True,
                ready_signal_in_log="Listening on 127.0.0.1:11434", # Sinal de prontidão
                event_to_set_on_ready=ollama_ready_event      # Evento a ser sinalizado
            ),
            run_process_wrapper(
                ['ollama', 'pull', 'gemma3:4b'],
                event_to_wait_for=ollama_ready_event          # Espera pelo ollama serve
            ),
            # Só executa ngrok http se ngrok_key estiver definida, caso contrário, o comando falharia
            run_process_wrapper(
                ['ngrok', 'http', '--log', 'stderr', '11434'],
                event_to_wait_for=ollama_ready_event,         # Também espera pelo ollama serve
                suppress_ngrok_key_in_cmd_print=False
            ) if ngrok_key else asyncio.sleep(0) # Se não houver chave, não faz nada com ngrok http
        )
    except asyncio.CancelledError:
        print("Execução principal (main gather) foi cancelada.")
    except Exception as e:
        print(f"Erro na execução principal (main gather): {e}")



# Para executar no Colab (ou em um ambiente com loop de eventos já rodando):
# await main()

# Ou, se for um script Python puro ou se o await main() não funcionar:
# asyncio.run(main())

# Exemplo de como chamar no Colab, lidando com loop de eventos existente:
# loop = asyncio.get_event_loop()
# if loop.is_running():
#     print("Loop de eventos asyncio já está rodando. Criando task para main().")
#     loop.create_task(main())
# else:
#     print("Iniciando novo loop de eventos asyncio para main().")
#     asyncio.run(main())

# (Todo o seu código anterior: imports, ngrok_key, ollama_ready_event, run_process_wrapper, main, etc. permanece o mesmo)
# ...
# async def main():
#    ... (corpo da sua função main) ...
# ...

# 👇 MODIFIQUE ESTA PARTE FINAL PARA USAR 'await main()' 👇
print("--- SCRIPT: Iniciando execução de main() com await ---")
try:
    await main() # A mudança principal é aqui!
except KeyboardInterrupt:
    print("--- SCRIPT: Execução interrompida pelo usuário (KeyboardInterrupt) ---")
except Exception as e:
    print(f"--- SCRIPT: Erro fatal ao executar main(): {e} ---")
    import traceback
    traceback.print_exc()
finally:
    print("--- SCRIPT: Execução de main() finalizada (ou tentou finalizar). ---")

--- DEBUG: Valor de ngrok_key carregado: '2xyQZ0B07aUR5DmPRtPLEWlRGa2_8163Tahir1m121RJqPtXh' ---
--- SCRIPT: Iniciando execução de main() com await ---
>>> starting ngrok config add-authtoken [NGROK_KEY_OCULTA]
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> 'ngrok config add-authtoken [NGROK_KEY_OCULTA]' finalizado com código 0.
>>> starting ollama serve
>>> 'ollama' esperando pelo evento de prontidão...
>>> 'ngrok' esperando pelo evento de prontidão...
[ollama stdout]: Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
[ollama stdout]: Your new public key is:
[ollama stdout]: 
[ollama stdout]: ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIAvz7C5ncrZUn/7Y154OBS+tLpmzdDa5Dxt7QUSxNsRP
[ollama stdout]: 
[ollama stderr]: time=2025-06-03T00:37:10.565Z level=INFO source=routes.go:1234 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH

In [5]:
!curl --version
!python3 --version
!pip3 --version

!sudo apt-get update
!sudo apt-get install -y pciutils
!sudo apt-get install -y lspci

!curl -fsSL https://ollama.com/install.sh | sh

!useradd -r -s /bin/false -m -d /usr/share/ollama ollama
!groupadd ollama # Pode já existir
!usermod -a -G ollama $(whoami) # Adiciona seu usuário atual ao grupo ollama
!chown -R ollama:ollama /usr/share/ollama # Garante permissões no diretório home do usuário ollama

curl 7.81.0 (x86_64-pc-linux-gnu) libcurl/7.81.0 OpenSSL/3.0.2 zlib/1.2.11 brotli/1.0.9 zstd/1.4.8 libidn2/2.3.2 libpsl/0.21.0 (+libidn2/2.3.2) libssh/0.9.6/openssl/zlib nghttp2/1.43.0 librtmp/2.3 OpenLDAP/2.5.17
Release-Date: 2022-01-05
Protocols: dict file ftp ftps gopher gophers http https imap imaps ldap ldaps mqtt pop3 pop3s rtmp rtsp scp sftp smb smbs smtp smtps telnet tftp 
Features: alt-svc AsynchDNS brotli GSS-API HSTS HTTP2 HTTPS-proxy IDN IPv6 Kerberos Largefile libz NTLM NTLM_WB PSL SPNEGO SSL TLS-SRP UnixSockets zstd
Python 3.11.12
pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubu

In [6]:
!nohup ollama serve > /dev/null 2>&1 &

In [8]:
!ollama pull gemma3:4b

In [9]:
!ollama list

NAME         ID              SIZE      MODIFIED      
gemma3:4b    a2af6cc3eb7f    3.3 GB    4 seconds ago    


In [10]:
!ollama run gemma3:4b

>>> Send a message (/? for help)oi, 
... tudo
...  bem
... ?
Oi! Tudo bem por aqui, e com você? 😊 Como posso te ajudar hoje?


>>> Send a message (/? for help)agor
... a vc
...  est
... a mu
... ito 
... mais
...  rap
... ida,
...  que
...  bom
... 
Que bom que você percebeu! A equipe do Google tem feito muitos aprimoramentos na minha velocidade e inteligência. É ótimo saber que você está notando a diferença! 😊

O que você gostaria de fazer agora?


>>> Send a message (/? for help)/bue
... 
Unknown command '/bue'. Type /? for help
>>> Send a message (/? for help)/bye
... 
